This notebook is part of the paper: **Combining Deep Learning with Physics Based Features in Explosion-Earthquake Discrimination** published at Geophysical Research Letters. 

This notebook contains the two branch model built in the paper, and the training of the model. 

In [ ]:
import json
import os
import timeit
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras import layers, models, Input

In [ ]:
filename = './demo_data.npz'

data = np.load(filename)

# the waveform data
X_waveform = data['X_waveform']
# physics parameters, can be multiple
X_param = data['X_param']
# y label
y = data['y']

print(f"The shapes: X_waveform: {X_waveform.shape}\n"
      f"The shapes: X_param: {X_param.shape}\n"
      f"The shapes: y: {y.shape}\n")

# input to the two branch model
X = [X_waveform, X_param]

Counter(y)

In [ ]:
def two_branch_model(branch1_input_shape = (2000, 1, 3),
                     branch2_input_shape = (1, ),
                     root_filters = 64,
                     pooling_size = (4, 1),
                     kernel_size = (3, 1),
                     n_layers = 2,
                     clip_filters = None,
                     activation = 'relu',
                     cnn_dropout = 0.3,
                     optimizer='adam',
                     loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
                     metrics=['acc']):
    
    # the proposed two-branch model
    # Note: In the paper, we also can add another branch that takes in
    # the spectrogram as input, it is very similar, so not repeat it here. 
    
    # Branch 1 - deep learning branch
    input_branch1 = Input(shape=branch1_input_shape)
        
    branch1 = layers.Conv2D(filters=root_filters,
         kernel_size=kernel_size,
         activation=activation, padding="same")(input_branch1)
    branch1 = layers.MaxPooling2D(pooling_size)(branch1)
    n_kernels = root_filters
    for i in range(n_layers):

        if clip_filters:
            if n_kernels > clip_filters:
                n_kernels = clip_filters
            else:
                n_kernels *= 2
        else:
            n_kernels *= 2
                
        branch1 = layers.Conv2D(filters=n_kernels,
         kernel_size=kernel_size,padding="same",
                         activation=activation)(branch1)
        branch1 = layers.Dropout(cnn_dropout)(branch1)
        branch1 = layers.MaxPooling2D(pooling_size)(branch1)

    # convert image to vector 
    branch1 = layers.Flatten()(branch1)
    
    branch1 = layers.Dense(128, activation=activation)(branch1)
    
    # Branch 2, the physics parameter branch, can be multiple parameters
    input_branch2 = layers.Input(shape=branch2_input_shape)
    branch2 = layers.Dense(128, activation=tf.keras.activations.relu)(input_branch2)

    # Concatenate the two branch features
    concat = layers.Concatenate()([branch1, branch2])

    # decision layer
    concat = layers.Dropout(0.5)(concat)
    concat = layers.Dense(100, activation=activation)(concat)
    output = layers.Dense(2, activation='softmax')(concat)

    model = models.Model(inputs=[input_branch1, input_branch2], outputs=[output])
    
    model.summary()
    model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)
    return model

In [ ]:
version = '1_0'
output_path = f'./trained_model/combined_CNN_ANN_model_ver{version}'
filepath = f'{output_path}/my_best_model.hdf5'

callbacks = [ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max'),
             EarlyStopping(monitor='val_acc',
                           mode='max',
                           patience=30),
             TensorBoard(log_dir=f"{output_path}/logs")]


In [ ]:
### Training
# settings
tf.keras.backend.clear_session()
mirrored_strategy = tf.distribute.MirroredStrategy()

# run the training and testing using multiple GPUs
with mirrored_strategy.scope():

    model = two_branch_model() 

    class_weights = class_weight.compute_class_weight(
        class_weight = 'balanced',classes=np.unique(y),y=y)
    class_weights = {0:class_weights[0], 1:class_weights[1]}
    print(f"The class weights are: {class_weights}")

    # record time and start training

    start_time = timeit.default_timer()
    history = model.fit(X, y, epochs=2, 
                    validation_split=0.2,
                    batch_size=256,
                    shuffle=True,
                    callbacks=callbacks,
                    class_weight=class_weights
                                )
    elapsed = timeit.default_timer() - start_time
    # finish training

    # save the model structure as figure
    tf.keras.utils.plot_model(model, to_file=f'{output_path}/model.png', show_shapes=True)

    # save the training curve
    plt.plot(history.history['acc'], label='training acc')
    plt.plot(history.history['val_acc'], label='validation acc')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(f'{output_path}/training_curve.png')
    plt.close()

    results = {}
    results['history_curve'] = history.history
    results['training_time_s'] = elapsed

    json.dump(results, open(f'{output_path}/history.json', 'w'))

    # load the trained model
    model = tf.keras.models.load_model(f'{output_path}/my_best_model.hdf5')

    print('Done saving the results!')


## Models for comparison purposes

In [ ]:
def model_ann(input_shape = (1, ),
                root_neurons = 16,
                dropout = 0.3,
                n_layers = 4,
                clip_neurons = None,
                activation='relu',
                output_class = 2,
                output_activation='softmax'
                ):
    # physics parameter branch for comparison purpose. 
    
    # build cnn layers
    inputs = Input(shape=input_shape)
    y = layers.Dense(root_neurons, activation=activation)(inputs)
    y = layers.Dropout(dropout)(y)
    
    n_neuron = root_neurons
    for i in range(n_layers):

        if clip_neurons:
            if n_neuron > clip_neurons:
                n_neuron = clip_neurons
            else:
                n_neuron *= 2
        else:
            n_neuron *= 2
                
        y = layers.Dense(n_neuron, activation=activation)(y)
        y = layers.Dropout(dropout)(y)

    outputs = layers.Dense(output_class, activation=output_activation)(y)
     # model building by supplying inputs/outputs
    baseline_model = models.Model(inputs=inputs, outputs=outputs)
    baseline_model.summary()
    baseline_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])
    return baseline_model

In [ ]:
def model_cnn(input_shape = (2000, 1, 3),
                kernel_size = (3, 1),
                pooling_size = (2, 1),
                root_filters = 16,
                clip_filters = None,
                dense_dropout = 0.3,
                cnn_dropout = 0.1,
                n_layers = 4,
                activation='relu',
                output_class = 2,
                output_activation='softmax'
                ):
    
    # deep learning branch model for comparison purpose. 
    
    # build cnn layers
    inputs = Input(shape=input_shape)

    y = layers.Conv2D(filters=root_filters,
         kernel_size=kernel_size,
         activation=activation, padding="same")(inputs)
    y = layers.MaxPooling2D(pooling_size)(y)
    n_kernels = root_filters
    for i in range(n_layers):

        if clip_filters:
            if n_kernels > clip_filters:
                n_kernels = clip_filters
            else:
                n_kernels *= 2
        else:
            n_kernels *= 2
                
        y = layers.Conv2D(filters=n_kernels,
         kernel_size=kernel_size,padding="same",
                         activation=activation)(y)
        y = layers.Dropout(cnn_dropout)(y)
        y = layers.MaxPooling2D(pooling_size)(y)

    # convert image to vector 
    y = layers.Flatten()(y)
    # dropout regularization
    y = layers.Dropout(dense_dropout)(y)
    y = layers.Dense(100, activation=activation)(y)
    outputs = layers.Dense(output_class, activation=output_activation)(y)
     # model building by supplying inputs/outputs
    baseline_model = models.Model(inputs=inputs, outputs=outputs)
    baseline_model.summary()
    baseline_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])
    return baseline_model